In [ ]:
import json
from datetime import datetime
from tqdm import tqdm
from shroom_classifier_lcel_baseline_v2 import ShroomClassifier

In [ ]:
def run_experiment(cp, reference, submission):
    data = json.load(open(reference, "r"))
    predictions = []
    for dp in tqdm(data):
        prediction = cp.classify(dp)
        predictions.append(prediction)
    json.dump(predictions, open(submission, "w+"))

In [ ]:
RUN = "temp-1.7"
MODEL = "gpt-3.5-turbo"
TEMP = 1.7
DATE = datetime.utcnow().date().isoformat()

In [ ]:
CLASSIFIER = ShroomClassifier(model_name=MODEL, temperature=TEMP)
run_experiment(CLASSIFIER, 'reference/val.model-agnostic.json', 'submission/val.model-agnostic.json')
run_experiment(CLASSIFIER, 'reference/val.model-aware.json', 'submission/val.model-aware.json')

In [ ]:
! python score.py --is_val submission reference scores/score-{MODEL}-{RUN}-{DATE}.txt
! mv submission/val.model-agnostic.json results/val.model-agnostic.{MODEL}-{RUN}-{DATE}
! mv submission/val.model-aware.json results/val.model-aware.{MODEL}-{RUN}-{DATE}